<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Устанавливаем нужные зависимости

In [1]:
!pip install onnx transformers onnxruntime

     |████████████████████████████████| 12.8 MB 4.9 MB/s 
     |████████████████████████████████| 4.0 MB 42.0 MB/s 
     |████████████████████████████████| 5.2 MB 25.0 MB/s 
     |████████████████████████████████| 6.5 MB 15.0 MB/s 
     |████████████████████████████████| 596 kB 35.2 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 895 kB 47.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Фиксируем версии библиотек

In [2]:
!pip freeze > req.txt

Импорты

In [3]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from typing import Mapping, OrderedDict
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
import onnx
import numpy as np

Запускаем rubert как есть

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [5]:
%%time
predict('Правительство выделит 16 миллиардов рублей на поддержку клещей')
#вернулся нейтральный класс

CPU times: user 106 ms, sys: 43 ms, total: 149 ms
Wall time: 282 ms


0

Переводим в ONNX
1. Есть библиотека transforms, где все из коробки

https://huggingface.co/docs/transformers/serialization - делаем все по лучшим гайдам

In [6]:
class DistilBertOnnxConfig(OnnxConfig):
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        return OrderedDict(
            [
                ("input_ids", {0: "batch", 1: "sequence"}),
                ("attention_mask", {0: "batch", 1: "sequence"}),
                ("token_type_ids", {0: "batch", 1: "sequence"}),
            ]
        )

In [7]:
config = AutoConfig.from_pretrained("blanchefort/rubert-base-cased-sentiment")
onnx_config_for_seq_clf = DistilBertOnnxConfig(config, task="sequence-classification")
print(onnx_config_for_seq_clf.outputs)

OrderedDict([('logits', {0: 'batch'})])


In [11]:
!mkdir -p output/onnx_transforms

In [12]:
onnx_inputs, onnx_outputs = export(
        tokenizer,
        model,
        onnx_config_for_seq_clf,
        output=Path("output/onnx_transforms/rubert.onnx"),
        opset=11)

Пробуем запустить в ONNX и посмотреть время инференса

In [13]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert.onnx", sess_options, providers)

Смотрим на входы и выходы импортированной модели

In [14]:
input_names = model_ONNX.get_inputs()
for input_name in input_names:
  print(input_name.name)

input_ids
attention_mask
token_type_ids


In [15]:
for on in model_ONNX.get_outputs():
  print(on.name)

logits


In [16]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  result  = model_ONNX.run(None, dict(inputs))
  predicted = result.index(max(result))
  return predicted

In [17]:
%%time
predict_onnx('Правительство выделит 16 миллиардов рублей на поддержку клещей')

CPU times: user 57.6 ms, sys: 1.11 ms, total: 58.7 ms
Wall time: 61.2 ms


0